### Mount Google Drive

Run the following cell to mount your Google Drive. You will be prompted to authorize Colab to access your Google Drive.

After mounting, you can navigate your Google Drive files in the left sidebar (file icon) to find the exact path to your 'Cat Species' folder.

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Setup and Dependencies

First, let's make sure all necessary libraries are imported and the device (CPU/GPU) is set.

In [13]:
import torch
import os
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights

# Set device for GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Model Initialization and Transformation Definition

This section initializes the pre-trained ResNet18 model and defines the image transformations needed before passing images to the model.

In [14]:
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

### ImageNet Class Labels and Cat Keywords

Here, we load the ImageNet class labels that the ResNet model was trained on and define keywords to identify 'cat' related predictions.

In [15]:
imagenet_classes = ResNet50_Weights.IMAGENET1K_V1.meta["categories"]

valid_cat_keywords = [
    "cat", "tabby", "tiger", "persian", "egyptian", "lynx", "siamese", "cheetah", "jaguar", "leopard", "lion", "puma"
]
print(f"Valid cat keywords: {valid_cat_keywords}")

Valid cat keywords: ['cat', 'tabby', 'tiger', 'persian', 'egyptian', 'lynx', 'siamese', 'cheetah', 'jaguar', 'leopard', 'lion', 'puma']


### Image Processing and Accuracy Calculation

Finally, this section iterates through the images in your 'Cat Species' directory, processes them with the model, and calculates a 'proxy accuracy' based on whether the top prediction contains a valid cat keyword. The `image_dir` is set to the path you provided from Google Drive.

In [16]:
image_dir = '/content/drive/MyDrive/Gen_Ai_Lab/Cat Species'

correct = 0
total = 0

if not os.path.exists(image_dir):
    print(f"Error: The directory '{image_dir}' does not exist. Please check the path.")
else:
    for species in os.listdir(image_dir):
        species_path = os.path.join(image_dir, species)

        if not os.path.isdir(species_path):
            continue

        for img_name in os.listdir(species_path):
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            img_path = os.path.join(species_path, img_name)
            try:
                image = Image.open(img_path).convert("RGB")
                image = transform(image).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = model(image)
                    pred_idx = output.argmax(dim=1).item()

                label = imagenet_classes[pred_idx].lower()
                print(f"Processing {img_name} (from {species}): Predicted label = {label}")

                if any(k in label for k in valid_cat_keywords):
                    correct += 1

                total += 1
            except Exception as e:
                print(f"Error processing image {img_name} in {species_path}: {e}")

    if total > 0:
        accuracy = (correct / total) * 100
        print(f"\nProxy Accuracy (Cat Detection): {accuracy:.2f}%")
    else:
        print("No valid images processed from the specified directory. Check the folder content and image types.")

Processing Abyssinian_1.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_2.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_3.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_4.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_5.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_6.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_7.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_8.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_9.png (from Abyssinian): Predicted label = egyptian cat
Processing Abyssinian_10.png (from Abyssinian): Predicted label = egyptian cat
Processing American Bobtail_1.png (from American Bobtail): Predicted label = egyptian cat
Processing American Bobtail_2.png (from American Bobtail): Predicted label = egyptian cat
Processing American Bobtail_3.png (from